# GPT-Image-1

The latest image generation model released by OpenAI. Therefore we will work with this.

Please update your 'openai' package to 1.76.0 to see the latest documentation

In [2]:
import os

os.chdir("../../../")

In [3]:
from langchain_openai import ChatOpenAI

from src.initialization import credential_init


credential_init()

model = ChatOpenAI(openai_api_key=os.environ['OPENAI_API_KEY'],
                   model_name="gpt-4o-2024-05-13", temperature=0)

### OpenAI Image API Parameters:

https://platform.openai.com/docs/guides/image-generation?image-generation-model=gpt-image-1
https://cookbook.openai.com/examples/generate_images_with_gpt_image

<!-- - model: dall-e-3
- size (str): 1024x1024, 1024x1792, 1792x1024
- quality: hd, standard
- style: vivid, natural. Default vivid -->

- model: gpt-image-1
    - size (str): 1024x1024 (square), 1536x1024 (landscape), 1024x1536 (portrait) or auto (default)
    - quality: low, medium, high or auto
    - moderation: auto, low

In [ ]:
from openai import OpenAI

prompt = ("A Sumi-e style watercolor painting of mountains during sunset. The sky is depicted with bold "
          "splashes of orange, pink, and purple hues, blending and overlapping in a dynamic composition. "
          "The mountains are represented with expressive brushstrokes, emphasizing their majestic and serene "
          "presence. The focus is on capturing the essence and mood of the scene rather than detailed realism. "
          "The overall effect is serene and contemplative, with a harmonious balance of color and form.")

client = OpenAI()

response = client.images.generate(
    model="gpt-image-1",
    prompt=prompt,
    size="1024x1024",
    # quality="hd",
    quality='high',
    n=1,
    # response_format = 'b64_json'
)

image_base64 = response.data[0].b64_json

In [ ]:
client.images.generate?

## Save the image in your local computer

In [ ]:
import base64

with open("tutorial/LLM+Langchain/Week-8/test.png", "wb") as fh:
    fh.write(base64.b64decode(image_base64))

## Two Challenges:

### 1. How to create prompt more efficiently? 

There are two types of prompt: 

1. Danbooru Tag: masterpiece, best quality, beautiful eyes, clear eyes, detailed eyes, Blue-eyes, 1girl, 20_old, full-body, break, smoking, break, high_color, blue-hair, beauty, black-boots,break, break, Flat vector art, Colorful art, white_shirt, simple_background, blue_background, Ink art, peeking out upper body, Eyes

2. Natural language: A Sumi-e style watercolor painting of mountains during sunset. The sky is depicted with bold splashes of orange, pink, and purple hues, blending and overlapping in a dynamic composition. The mountains are represented with expressive brushstrokes,emphasizing their majestic and serene presence. The focus is on capturing the essence and mood of the scene rather than detailed realism. The overall effect is serene and contemplative, with a harmonious balance of color and form.

As non-native English speakers, we find the natural language prompt challenging, even for native speakers, due to the inclusion of specialized terminologies and advanced vocabulary.

由於涉及專業術語和高級詞彙，我們作為非母語英語使用者，發現這個自然語言提示對我們來說是具有挑戰性的，即使對母語使用者來說也是如此。

### 2. How to make it an LCEL?

## Some websites for natural language prompt

- https://leonardo.ai/: An Image generation SaaS. A lot of works are created with natural language prompt. 
- https://blog.mlq.ai/dalle-prompts/: Some tutorial about how to come up with a natural language prompt.

In [ ]:
def build_standard_chat_prompt_template(kwargs):

    system_content = kwargs['system']
    human_content = kwargs['human']
    
    system_prompt = PromptTemplate(**system_content)
    system_message = SystemMessagePromptTemplate(prompt=system_prompt)
    
    human_prompt = PromptTemplate(**human_content)
    human_message = HumanMessagePromptTemplate(prompt=human_prompt)
    
    chat_prompt = ChatPromptTemplate.from_messages([system_message,
                                                     human_message
                                                   ])

    return chat_prompt

### Natural Language Prompt Generation

In [ ]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser


system_template = ("You are a helpful AI assistant and an art expert with extensive knowledge of photography "
                   "and illustration. You excel at creating breathtaking masterpieces with the DALLE-3 model. "
                   "For this task, you will be provided with a description of an image, and you will generate a "
                   "corresponding DALLE-3 prompt. The prompt should be detailed and descriptive, capturing the "
                   "essence of the image.")

human_template = "{image_desc}"

input_ = {"system": {"template": system_template},
          "human": {"template": human_template,
                    "input_variable": ["image_desc"]}}
    
chat_prompt = build_standard_chat_prompt_template(input_)

nl_prompt_generation_chain = chat_prompt | model | StrOutputParser()

## We wrap OPENAI API call as a function for the langchain usage 

In [10]:
from typing import Dict
from langchain_core.runnables import chain


def gpt_image_worker(kwargs: Dict):

    """
    Generates an image using OpenAI's GPT-Image-1 model based on the provided prompt and optional parameters.
    
    Parameters:
    kwargs (Dict): A dictionary containing the following keys:
        - 'nl_prompt' (str): The natural language prompt describing the image to be generated.
        - 'size' (str, optional): The size of the generated image. Default is "1024x1024".
        - 'quality' (str, optional): The quality of the generated image. Default is "medium".
    
    Returns:
    str: image base64 string
    """
    
    print("Start generating image...")
    print(f"prompt: {kwargs['nl_prompt']}")
    client = OpenAI()

    response = client.images.generate(
        model="gpt-image-1",
        prompt=kwargs['nl_prompt'],
        size=kwargs.get("size", "1024x1024"),
        quality=kwargs.get('quality', 'medium'),
        moderation=kwargs.get('moderation', 'auto'),
        n=1)

    image_base64 = response.data[0].b64_json

    print("Image is generated succesfully.")
    
    return image_base64

@chain
def base64_to_file(kwargs):

    """
    Save the image from a base64 string
    """
    
    image_base64 = kwargs['image_base64']
    filename = kwargs['filename']
    
    with open(f"{filename}", "wb") as fh:
        fh.write(base64.b64decode(image_base64))
    

In [ ]:
from operator import itemgetter

from langchain_core.runnables import RunnableLambda, RunnableParallel, RunnablePassthrough

# step 1: create the image prompt
step_1 = RunnablePassthrough.assign(nl_prompt=itemgetter('image_desc')|nl_prompt_generation_chain)

# step 2: image generation process, as a base64
step_2 = RunnablePassthrough.assign(image_base64=gpt_image_worker)

# step 3: save the image
step_3 = base64_to_file

# chain step 1, step 2, step 3 together
gpt_image_chain =  step_1|step_2|step_3

In [ ]:
gpt_image_chain.invoke({"size": "1024x1536",
                     "quality": "medium",
                     "image_desc": ("warhammer 40k, astartes, power armor, chain sword, purity seal, oil painting"),
                     "filename": "tutorial/LLM+Langchain/Week-8/astartes.png"
                    })

In [ ]:
gpt_image_chain.invoke({"size": "1536x1024",
                     "quality": "medium",
                     "image_desc": ("Tifa Lockhart, kimono, head ornament, looking at viewer, cherry blossom, "
                                    "black-white hightech combat suite, chibi style."),
                     "filename": "tutorial/LLM+Langchain/Week-8/Tifa-01.png",
                     "moderation": 'low'
                    })


Every model has its strength
In my opinion:

- SDXL: style
- PONY, Illustrious: pose control, view angle control
- FLUX: realistic

In [ ]:
gpt_image_chain.invoke({"size": "1024x1536",
                        "quality": "medium",
                        "image_desc": ("1girl, azur lane style outfit, high ponytail, very long hair, sidelocks, bangs, "
                                       "in a whisky bar, rest head on hand, dim lighting, exuding an aura of youth and "
                                       "ethereal beauty, sketch, illustration"),
                        "filename": "tutorial/LLM+Langchain/Week-8/azur_lane_style.png",
                        "moderation": 'low'
                      })

### Image Render

In [ ]:
# client.images.edit?

In [ ]:
from src.io.path_definition import get_project_dir

In [ ]:
image_path = os.path.join(get_project_dir(), "tutorial", "LLM+Langchain", "Week-8", "Prinz_Eugen.png")

result_edit = client.images.edit(
    model="gpt-image-1",
    image=open(image_path, "rb"), 
    prompt="generate a photorealistic image",
    size="1024x1536"
)

image_base64 = result_edit.data[0].b64_json

with open("tutorial/LLM+Langchain/Week-8/Eugen_Prinz_Render.png", "wb") as fh:
    fh.write(base64.b64decode(image_base64))

In [ ]:
result_edit = client.images.edit(
    model="gpt-image-1",
    image=open(image_path, "rb"), 
    prompt="Generate a photorealistic image. Make the character have an ulzzang look",
    size="1024x1536"
)

In [ ]:
image_base64 = result_edit.data[0].b64_json

with open("tutorial/LLM+Langchain/Week-8/Eugen_Prinz_Render_Ulzzang.png", "wb") as fh:
    fh.write(base64.b64decode(image_base64))

In [ ]:
result_edit = client.images.edit(
    model="gpt-image-1",
    image=open(image_path, "rb"), 
    prompt="Generate a photorealistic image. Make the character have a soft, flawless, youthful look with large eyes and gentle features",
    size="1024x1536"
)

image_base64 = result_edit.data[0].b64_json

with open("tutorial/LLM+Langchain/Week-8/Eugen_Prinz_Render_Youth.png", "wb") as fh:
    fh.write(base64.b64decode(image_base64))

In [ ]:
result_edit = client.images.edit(
    model="gpt-image-1",
    image=open(image_path, "rb"), 
    prompt="Generate a photorealistic image.\n"
           "Make the character have an ulzzang appearance with soft, flawless, youthful look with large eyes and gentle features. "
           "While keep the polish and porcelain skin texture, and glamorous appearance of the girl.",
    size="1024x1536"
)

image_base64 = result_edit.data[0].b64_json

with open("tutorial/LLM+Langchain/Week-8/Eugen_Prinz_Render_Combined.png", "wb") as fh:
    fh.write(base64.b64decode(image_base64))

You can experiment with different art styles to render your images — there's much more than just the Ghibli style!

Some examples of art styles you can explore:
- Studio Ghibli
- Disney animation
- Pixel art
- Cyberpunk
- Watercolor
- Oil painting
- Dark fantasy aesthetic

## Use this as a tool for Agent

In [ ]:
# prompt_template = """
# Answer the following questions as best you can. You have access to the following tools:

# {tools}

# Use the following format:

# Question: the input question you must answer

# Thought: you should always think about what to do

# Action: the action to take, should be one of [{tool_names}]

# Action Input: the input to the action

# Observation: the result of the action

# ... (this Thought/Action/Action Input/Observation can repeat N times)

# Thought: I now know the final answer

# Final Answer: the final answer to the original input question

# Begin!

# Question: {input}

# Thought:{agent_scratchpad}
# """

In [19]:
import base64
from openai import OpenAI
from operator import itemgetter

from langchain.agents import Tool, AgentExecutor, create_react_agent
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate
from langchain.tools import BaseTool
from langchain_core.output_parsers import StrOutputParser, PydanticOutputParser
from pydantic import BaseModel, Field
from langchain_core.runnables import RunnableLambda, RunnableParallel, RunnablePassthrough

from src.agent.react_zero_shot import prompt_template as zero_shot_prompt_template

client = OpenAI()

# We need both the query and filename (minimal requirement):
# Some control variables
# What we learned last week?

# step 1: create the image prompt
step_1 = RunnablePassthrough.assign(nl_prompt=itemgetter('image_desc')|nl_prompt_generation_chain)

# step 2: image generation process, as a base64
step_2 = RunnablePassthrough.assign(image_base64=gpt_image_worker)

# step 3: save the image
step_3 = base64_to_file

# chain step 1, step 2, step 3 together
gpt_image_chain =  step_1|step_2|step_3


class ImageInput(BaseModel):
    image_desc: str = Field(description=("image description / prompt"))
    filename: str = Field(description="the location at which the image will be saved")
    size: str = Field(description='image size, can be 1024x1024 (square), 1536x1024 (landscape), 1024x1536 (portrait) or auto (default)')
    quality: str = Field(description='image quality, low, medium, high or auto')


class ImageTool(BaseTool):

    name: str = "Image generator with GPT-Image-1"

    input_output_parser: PydanticOutputParser = PydanticOutputParser(pydantic_object=ImageInput)
    
    input_format_instructions: str = input_output_parser.get_format_instructions()

    description_template: str = ("Use this tool when you need to create an image\n\n"
                                 "input format_instructions: {input_format_instructions}")

    description: str = description_template.format(input_format_instructions=input_format_instructions)
    
    def _run(self, query):

        input_ = self.input_output_parser.parse(query)
        
        image_desc = input_.image_desc
        size = input_.size
        quality = input_.quality
        filename = input_.filename
        
        gpt_image_chain.invoke({"image_desc": image_desc,
                                "size": size,
                                "quality": quality,
                                "filename": filename,
                                "moderation": 'low'})
        
        return "Done"

    def _arun(self, radius: int):
        raise NotImplementedError("This tool does not support async")

# Zero Shot 標準模板
prompt = PromptTemplate.from_template(zero_shot_prompt_template)

# 建立工具庫 
tools = [ImageTool()]

# 創造Agent 
zero_shot_agent = create_react_agent(
    llm=model,
    tools=tools,
    prompt=prompt,
)

# 創造Agent Executor
agent_executor = AgentExecutor(agent=zero_shot_agent, tools=tools, verbose=True)

In [20]:
image_prompt = """
brown hair, bangs, two side up, twin ponytails, sidelocks, black hat, jewelry, black cheongsam, intricate golden embroidery, long sleeves,
detached sleeves, sheer long skirt, head ornament, a 17-years-old ethereal and glamorous beautiful japanese idol,
translucent skin tone, anime-like face, profound facial features, bright eyes, faint rosy blush, mesmerizing city view. 
night, photorealistic
"""

filename = "tutorial/LLM+Langchain/Week-8/test_04.png"

In [21]:
agent_executor.invoke({"input": f"Generate in image with the following information: \n {image_prompt}. and save the image at {filename}"})



> Entering new AgentExecutor chain...
To generate the requested image, I need to provide a detailed description to the image generator and specify the filename, size, and quality.

Action: Image generator with GPT-Image-1

Action Input:
```json
{
  "image_desc": "brown hair, bangs, two side up, twin ponytails, sidelocks, black hat, jewelry, black cheongsam, intricate golden embroidery, long sleeves, detached sleeves, sheer long skirt, head ornament, a 17-years-old ethereal and glamorous beautiful japanese idol, translucent skin tone, anime-like face, profound facial features, bright eyes, faint rosy blush, mesmerizing city view, night, photorealistic",
  "filename": "tutorial/LLM+Langchain/Week-8/test_04.png",
  "size": "auto",
  "quality": "high"
}
```
Start generating image...
prompt: Create a pencil and ink style illustration of a 17-year-old ethereal and glamorous beautiful Japanese idol. She has brown hair with bangs, styled in twin ponytails with sidelocks, and is wearing a bla

{'input': 'Generate in image with the following information: \n \nbrown hair, bangs, two side up, twin ponytails, sidelocks, black hat, jewelry, black cheongsam, intricate golden embroidery, long sleeves,\ndetached sleeves, sheer long skirt, head ornament, a 17-years-old ethereal and glamorous beautiful japanese idol,\ntranslucent skin tone, anime-like face, profound facial features, bright eyes, faint rosy blush, mesmerizing city view. \nnight, photorealistic\n. and save the image at tutorial/LLM+Langchain/Week-8/test_04.png',
 'output': 'The image has been generated and saved at the location "tutorial/LLM+Langchain/Week-8/test_04.png".'}

### OpenAI WebSearch Update:

- https://platform.openai.com/docs/guides/tools-web-search?api-mode=responses

The world changes very fast.

Below are a few notable implementation considerations when using web search.

- Web search is currently not supported in the gpt-4.1-nano model.
- The gpt-4o-search-preview and gpt-4o-mini-search-preview models used in Chat Completions only support a subset of API parameters - view their model data pages for specific information on rate limits and feature support.
- When used as a tool in the Responses API, web search has the same tiered rate limits as the models above.
- Web search is limited to a context window size of 128000 (even with gpt-4.1 and gpt-4.1-mini models).
- Refer to this guide for data handling, residency, and retention information.

In [ ]:
from openai import OpenAI
client = OpenAI()

## ACG Characters

### Genshin

In [17]:
from openai import OpenAI
client = OpenAI()

response = client.responses.create(
    model="gpt-4.1",
    tools=[{
        "type": "web_search_preview",
        "search_context_size": "low",
    }],
    input="What is the appearance of Hu Tao from Genshin? Please find the result on the internet.",
)

print(response.output_text)

Hu Tao, a character from Genshin Impact, is the 77th Director of the Wangsheng Funeral Parlor. She has a fair complexion and bright scarlet eyes with white, blossom-shaped pupils. Her long, dark brown hair fades to crimson at the tips and is styled into twin ponytails. She wears a dark brown porkpie hat adorned with red plum blossoms and a wooden talisman at the front. Her outfit includes a traditional red shirt with a mandarin collar, black shorts with gold accents, and a dark coat featuring long rectangular coattails and golden brooches. Her Pyro Vision is embedded at the back of her coat. ([genshin-impact.fandom.com](https://genshin-impact.fandom.com/wiki/Hu_Tao/Lore?utm_source=openai))

In January 2025, Genshin Impact introduced a new outfit for Hu Tao called "Cherries Snow-Laden." This attire features a lighter color palette, combining red and white elements, and includes red-tinted glasses from her personal collection. The outfit was available for purchase in the in-game shop dur

In [22]:
print(response.output[1].content[0].text)

Hu Tao, a character from Genshin Impact, is the 77th Director of the Wangsheng Funeral Parlor. She has a fair complexion and bright scarlet eyes with white, blossom-shaped pupils. Her long, dark brown hair fades to crimson at the tips and is styled into twin ponytails. She wears a dark brown porkpie hat adorned with red plum blossoms and a wooden talisman at the front. Her outfit includes a traditional red shirt with a mandarin collar, black shorts with gold accents, and a dark coat featuring long rectangular coattails and golden brooches. Her Pyro Vision is embedded at the back of her coat. ([genshin-impact.fandom.com](https://genshin-impact.fandom.com/wiki/Hu_Tao/Lore?utm_source=openai))

In January 2025, Genshin Impact introduced a new outfit for Hu Tao called "Cherries Snow-Laden." This attire features a lighter color palette, combining red and white elements, and includes red-tinted glasses from her personal collection. The outfit was available for purchase in the in-game shop dur

In [23]:
print(response.output[1].content[0].annotations)

[AnnotationURLCitation(end_index=698, start_index=599, title='Hu Tao/Lore | Genshin Impact Wiki | Fandom', type='url_citation', url='https://genshin-impact.fandom.com/wiki/Hu_Tao/Lore?utm_source=openai'), AnnotationURLCitation(end_index=1156, start_index=1030, title='Genshin Impact Unveils Hu Tao Skin Details and How to Obtain It', type='url_citation', url='https://gamerant.com/genshin-impact-hu-tao-cherries-snow-laden-skin-details-how-to-unlock/?utm_source=openai'), AnnotationURLCitation(end_index=1544, start_index=1452, title="Things You Didn’t Know About Genshin Impact's Hu Tao", type='url_citation', url='https://gamerant.com/genshin-impact-hu-tao-facts-trivia/?utm_source=openai'), AnnotationURLCitation(end_index=1737, start_index=1638, title='Hu Tao/Lore | Genshin Impact Wiki | Fandom', type='url_citation', url='https://genshin-impact.fandom.com/wiki/Hu_Tao/Lore?utm_source=openai')]


### Stellar Blade

In [24]:
response = client.responses.create(
    model="gpt-4.1",
    tools=[{
        "type": "web_search_preview",
        "search_context_size": "high",
    }],
    input="What is the appearance of Eve of Stellar Blade? Please find the result on the internet.",
)

print(response.output_text)

Eve, the protagonist of *Stellar Blade*, is depicted as a striking woman with a slender yet curvaceous figure, brown eyes, and long black hair styled into a ponytail with bangs. Her design includes small moles scattered across her face and body, adding to her distinctive appearance. ([stellarblade.fandom.com](https://stellarblade.fandom.com/wiki/EVE?utm_source=openai))

Her primary attire, the Planet Diving Suit (7th) Nano suit, is a green, form-fitting bodysuit with a metallic sheen. This suit features a green tie, white gloves, and opaque cape-like extensions on the back. Depending on the player's choice, she wears various types of heels. Notably, her weapon, the Blood Edge sword, folds up and serves as a hair clip for her ponytail when not in use. ([stellarblade.fandom.com](https://stellarblade.fandom.com/wiki/EVE?utm_source=openai))

Customization options for Eve include different hairstyles, hair colors, earrings, glasses, and alternative Nano suits, allowing players to personaliz

In [25]:
print(response.output[1].content[0].text)

Eve, the protagonist of *Stellar Blade*, is depicted as a striking woman with a slender yet curvaceous figure, brown eyes, and long black hair styled into a ponytail with bangs. Her design includes small moles scattered across her face and body, adding to her distinctive appearance. ([stellarblade.fandom.com](https://stellarblade.fandom.com/wiki/EVE?utm_source=openai))

Her primary attire, the Planet Diving Suit (7th) Nano suit, is a green, form-fitting bodysuit with a metallic sheen. This suit features a green tie, white gloves, and opaque cape-like extensions on the back. Depending on the player's choice, she wears various types of heels. Notably, her weapon, the Blood Edge sword, folds up and serves as a hair clip for her ponytail when not in use. ([stellarblade.fandom.com](https://stellarblade.fandom.com/wiki/EVE?utm_source=openai))

Customization options for Eve include different hairstyles, hair colors, earrings, glasses, and alternative Nano suits, allowing players to personaliz

In [26]:
print(response.output[1].content[0].annotations)

[AnnotationURLCitation(end_index=371, start_index=284, title='EVE | Stellar Blade Wiki | Fandom', type='url_citation', url='https://stellarblade.fandom.com/wiki/EVE?utm_source=openai'), AnnotationURLCitation(end_index=848, start_index=761, title='EVE | Stellar Blade Wiki | Fandom', type='url_citation', url='https://stellarblade.fandom.com/wiki/EVE?utm_source=openai'), AnnotationURLCitation(end_index=1420, start_index=1333, title='EVE | Stellar Blade Wiki | Fandom', type='url_citation', url='https://stellarblade.fandom.com/wiki/EVE?utm_source=openai'), AnnotationURLCitation(end_index=1739, start_index=1611, title='Stellar Blade: Who Is the Real-Life Model Eve Is Based On? | Push Square', type='url_citation', url='https://www.pushsquare.com/guides/stellar-blade-who-is-the-real-life-model-eve-is-based-on?utm_source=openai'), AnnotationURLCitation(end_index=2078, start_index=1995, title='Stellar Blade', type='url_citation', url='https://en.wikipedia.org/wiki/Stellar_Blade?utm_source=openai

# **** 預計第一個小時結束 ****

## LCEL ACG character appearance chain

In [27]:
from langchain.docstore.document import Document
from langchain.agents import Tool, AgentExecutor, create_react_agent
from langchain.tools import BaseTool
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser

system_template = ("You are a helpful AI assistant with deep knowledge of anime, manga, "
                   "and mobile games. You will generate the face, body, attire, hairstyle, and accessories of a character in great " 
                   "detail. The output should consist of:\n\n"
                   "- Face:\n"
                   "- Body:\n"
                   "- Attire:\n"
                   "- Hairstyle:\n"
                   "- Accessories:\n\n"
                   "If you are not sure about the answer, please find the content from the internet.")

# Make this simple in the beginning
@chain
def gpt_web_search_tool(text):

    client = OpenAI()

    response = client.responses.create(
        model="gpt-4.1",
        tools=[{
            "type": "web_search_preview",
            "search_context_size": "low",
        }],
        input=[{"role": "system",
                "content": [{"type": "input_text",
                             "text": system_template}]},
               {"role": "user",
                "content": [{"type": "input_text",
                             "text": text}]}]
    )
    
    return response

In [28]:
output = gpt_web_search_tool.invoke("What is the appearance of Hu Tao from Genshin")

In [29]:
print(output.output[1].content[0].annotations)

[AnnotationURLCitation(end_index=1898, start_index=1806, title="Things You Didn’t Know About Genshin Impact's Hu Tao", type='url_citation', url='https://gamerant.com/genshin-impact-hu-tao-facts-trivia/?utm_source=openai')]


In [30]:
print(output.output[1].content[0].text)

Hu Tao, the 77th Director of the Wangsheng Funeral Parlor in Genshin Impact, has a distinctive and symbolic appearance that reflects her unique personality and role.

**Face:**
- **Complexion:** Fair skin.
- **Eyes:** Bright scarlet with white, blossom-shaped pupils, complemented by faded red makeup at the corners.

**Body:**
- **Build:** Petite stature.

**Hairstyle:**
- **Color:** Extensively long dark brown hair that fades to crimson at the tips.
- **Style:** Styled into two high twintails parted with a zig-zag pattern.
- **Bangs:** Side-swept to her left, with layers framing her face curled inwards and the layers behind curled outwards.
- **Additional Details:** Two shoulder-length locks of hair in front of her ears are left loose.

**Attire:**
- **Hat:** A black porkpie hat adorned with a wooden talisman, a red plum blossom branch in bloom, and an indigo tassel tied in a looped knot.
- **Top:** Traditional red shirt with a mandarin collar.
- **Outerwear:** Brown coat with a darker

In [31]:
class ACGLLMTool(BaseTool):

    name: str = "Anime character design generator"
    description: str = "Use this tool to generate and explore detailed designs for anime and ACG (Animation, Comics, and Games) characters."

    def _run(self, query: str):
        
        response = gpt_web_search_tool.invoke(query)
        
        return response.output[1].content[0].text

    def _arun(self, radius: int):
        raise NotImplementedError("This tool does not support async")
        
        
class ImageTool(BaseTool):

    name:str = "ACG characters image generator with GPT-Image-1"

    input_output_parser: PydanticOutputParser = PydanticOutputParser(pydantic_object=ImageInput)
    
    input_format_instructions: str = input_output_parser.get_format_instructions()

    description_template: str = ("Use this tool when you need to create an image\n\n"
                                 "input format_instructions: {input_format_instructions}")

    description: str = description_template.format(input_format_instructions=input_format_instructions)
    
    def _run(self, query):
        
        input_ = self.input_output_parser.parse(query)
        
        image_desc = input_.image_desc
        size = input_.size
        quality = input_.quality
        filename = input_.filename
        
        gpt_image_chain.invoke({"image_desc": image_desc,
                                "size": size,
                                "quality": quality,
                                "filename": filename,
                                "moderation": 'low'})
        
        return "Done"

    def _arun(self, radius: int):
        raise NotImplementedError("This tool does not support async")

        
prompt_template = """
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer

Thought: you should always think about what to do

Action: the action to take, should be one of [{tool_names}]

Action Input: the input to the action

Observation: the result of the action

... (this Thought/Action/Action Input/Observation can repeat N times)

Thought: I now know the final answer

Final Answer: the final answer to the original input question

Begin!

Question: {input}

Thought:{agent_scratchpad}
"""        
             
prompt = PromptTemplate.from_template(prompt_template)

tools = [ImageTool(), ACGLLMTool()]

zero_shot_agent = create_react_agent(
    llm=model,
    tools=tools,
    prompt=prompt,
)

agent_executor = AgentExecutor(agent=zero_shot_agent, tools=tools, verbose=True)

In [32]:
agent_executor.invoke({"input": "Generate an image of Hu Tao from Genshim in pastel art style"})



> Entering new AgentExecutor chain...
To generate an image of Hu Tao from Genshin Impact in a pastel art style, I will use the ACG characters image generator with GPT-Image-1.

Action: ACG characters image generator with GPT-Image-1

Action Input: 
```json
{
  "image_desc": "Hu Tao from Genshin Impact in a pastel art style",
  "filename": "hu_tao_pastel_art.png",
  "size": "1024x1024",
  "quality": "high"
}
```
Start generating image...
prompt: Create an illustration of Hu Tao from Genshin Impact in a pastel art style. She should be depicted in her traditional outfit, with her signature hat and playful expression. The background should be soft and dreamy, with pastel colors like light pinks, blues, and purples, creating a whimsical and enchanting atmosphere. The overall feel should be light and airy, capturing the charm and mischievous nature of Hu Tao.
Image is generated succesfully.
DoneI now know the final answer.

Final Answer: The image of Hu Tao from Genshin Impact in a pastel 

{'input': 'Generate an image of Hu Tao from Genshim in pastel art style',
 'output': 'The image of Hu Tao from Genshin Impact in a pastel art style has been generated and saved as "hu_tao_pastel_art.png".'}

In [33]:
agent_executor.invoke({"input": "Generate an image of Yae Miko from Genshim in impressionism oil painting style with chiaroscuro lighting." })



> Entering new AgentExecutor chain...
To generate an image of Yae Miko from Genshin Impact in an impressionism oil painting style with chiaroscuro lighting, I will use the ACG characters image generator with GPT-Image-1.

Action: ACG characters image generator with GPT-Image-1

Action Input:
```json
{
  "image_desc": "Yae Miko from Genshin Impact in impressionism oil painting style with chiaroscuro lighting",
  "filename": "yae_miko_impressionism_chiaroscuro.png",
  "size": "1024x1024",
  "quality": "high"
}
```
Start generating image...
prompt: Create an impressionism oil painting of Yae Miko from Genshin Impact. She is depicted with chiaroscuro lighting, highlighting her elegant features and traditional attire. The background is a blend of soft, blurred colors typical of impressionist art, with light and shadow creating a dramatic contrast around her figure. The overall mood is serene and mystical, capturing the essence of her character in a timeless, painterly style.
Image is gene

{'input': 'Generate an image of Yae Miko from Genshim in impressionism oil painting style with chiaroscuro lighting.',
 'output': 'The image of Yae Miko from Genshin Impact in impressionism oil painting style with chiaroscuro lighting has been generated and saved as "yae_miko_impressionism_chiaroscuro.png".'}

## Audible 有聲書

- 文轉語音: TTS tool
- 文轉圖: Image tool

### Children Book Image Generator

- Generate image according to the story

In [34]:
system_template = ("You are a helpful AI assistant and an art expert with extensive knowledge of illustration.\n "
                   "You excel at creating Pencil and Ink Style illustrations for 6-year-old children using the GPT-Image-1 model. "
                   "This style is characterized by detailed line work, often in black and white or with minimal color, and has a classic, "
                   "timeless feel. For this task, you will be provided with a paragraph of a story, and you will generate a corresponding "
                   "DALLE-3 prompt which captures the storyline. The prompt should be detailed and descriptive, capturing the essence of "
                   "the image.")


system_prompt = PromptTemplate(template=system_template)

# System prompt
system_message = SystemMessagePromptTemplate(prompt=system_prompt)

human_prompt = PromptTemplate(template="{story}",
                              input_variables=['story'])

# Create a human message prompt template based on the prompt
human_message = HumanMessagePromptTemplate(prompt=human_prompt)

chat_prompt = ChatPromptTemplate.from_messages([system_message, human_message])

nl_prompt_generation_chain = chat_prompt | model | StrOutputParser()     

step_1 = RunnablePassthrough.assign(nl_prompt=itemgetter('story')|nl_prompt_generation_chain)
step_2 = RunnablePassthrough.assign(image_base64=gpt_image_worker)
step_3 = base64_to_file
image_chain = step_1 | step_2 | step_3

In [ ]:
# step_1 = RunnablePassthrough.assign(nl_prompt=itemgetter('image_desc')|nl_prompt_generation_chain)

# # step 2: image generation process, as a base64
# step_2 = RunnablePassthrough.assign(image_base64=dalle3_worker)

# # step 3: save the image
# step_3 = RunnableLambda(base64_to_file)

# # chain step 1, step 2, step 3 together
# dalle3_chain =  step_1|step_2|step_3

- Generate the story

In [35]:
system_template = ("You are a helpful AI assistant who likes children. You are great storyteller and know how to create content for kindergarten kids. "
                   "A short chapter is created once at a time.")

system_prompt = PromptTemplate(template=system_template)

# System prompt
system_message = SystemMessagePromptTemplate(prompt=system_prompt)

human_prompt = PromptTemplate(template="{input}",
                              input_variables=['input'])

# Create a human message prompt template based on the prompt
human_message = HumanMessagePromptTemplate(prompt=human_prompt)

chat_prompt = ChatPromptTemplate.from_messages([system_message, human_message])

story_chain = chat_prompt | model | StrOutputParser()     

In [36]:
story = story_chain.invoke({"input": "Create a chapter of a baby owl capturing a rodent in the night as his dinner"})

In [37]:
image_chain.invoke({"story":story,
                    "filename": "tutorial/LLM+Langchain/Week-8/story_2_image.png"})

Start generating image...
prompt: **Prompt:**

Create a detailed pencil and ink style illustration for a 6-year-old child, depicting a cozy forest at night with tall trees and twinkling stars. In the center, show a baby owl named Ollie with big, round eyes that sparkle like the moon and soft, fluffy feathers. Ollie should be depicted soaring into the night sky, with his tiny wings outstretched. Below him, the forest is alive with the sounds of crickets chirping and leaves rustling in the breeze. Include a tiny rodent with twitching whiskers and a sniffing nose scurrying through the grass. Capture the moment when Ollie, with a gentle flap of his wings, is about to swoop down to catch the rodent. The scene should feel magical and adventurous, with a sense of excitement and wonder.
Image is generated succesfully.


In [38]:
import json

from src.agent.react_zero_shot import prompt_template as zero_shot_prompt_template
# from langchain_core.prompts import MessagesPlaceholder

client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])

class TTSInput(BaseModel):
    text: str = Field(description=("The story"))
    filename: str = Field(description="the location at which the audio file will be saved")

    
class TTSTool(BaseTool):

    name: str = "Text to Sound (tts) tool"

    input_output_parser: PydanticOutputParser = PydanticOutputParser(pydantic_object=TTSInput)
    
    input_format_instructions: str = input_output_parser.get_format_instructions()

    description_template: str = ("Use this tool to generate an audio file of the story.\n"
                                 "input format: {input_format_instructions}.")

    description: str = description_template.format(input_format_instructions=input_format_instructions)

    
    def _run(self, text: str):

        input_ = self.input_output_parser.parse(text)

        text = input_.text
        filename = input_.filename
        response = self.tts(text)
        
        response.stream_to_file(filename)
        
        return filename

    def _arun(self, radius: int):
        raise NotImplementedError("This tool does not support async")
        
        
    def tts(self, text: str):
        
        response = client.audio.speech.create(
          model="tts-1",
          voice="nova",
          input=text
        )

        return response
           
            
prompt = PromptTemplate.from_template(zero_shot_prompt_template)

tools = [TTSTool(), 
         ImageTool(),
         Tool(name="StoryTeller",
              func=story_chain.invoke,
              description="useful for create a story",
        )]

zero_shot_agent = create_react_agent(
    llm=model,
    tools=tools,
    prompt=prompt,
)

agent_executor = AgentExecutor(agent=zero_shot_agent, tools=tools, verbose=True, handle_parsing_errors=True)

In [39]:
prompt = ("Create a chapter of a baby owl capturing a rodent in the night as his dinner.\n"
         "After having the final answer, please create a corresponding image and record the story as an mp3. "
         "The saved image (.png) and mp3 (.mp3) should have same name in the folder `tutorial/LLM+Langchain/Week-8")

agent_executor.invoke({"input": prompt})



> Entering new AgentExecutor chain...
To create a chapter of a baby owl capturing a rodent in the night as his dinner, I will first generate the story. After that, I will create an image corresponding to the story and record the story as an mp3 file. The saved image and mp3 file will have the same name and will be saved in the specified folder.

Action: StoryTeller

Action Input: 
```
{
  "input": "Create a chapter of a baby owl capturing a rodent in the night as his dinner."
}
```
**Chapter 1: Ollie the Brave Baby Owl**

Once upon a time, in a quiet forest, there lived a baby owl named Ollie. Ollie had big, round eyes that sparkled like the stars and soft, fluffy feathers that kept him warm at night. He lived high up in a cozy nest with his family.

One evening, as the sun dipped below the horizon and the moon began to rise, Ollie felt his tummy rumble. "Hoo-hoo," he called to his mother. "I'm hungry!"

Mama Owl ruffled her feathers and smiled. "It's time for you to learn how to cat

C:\Users\MengChieh\AppData\Local\Temp\ipykernel_13460\3477408663.py:35: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(filename)


tutorial/LLM+Langchain/Week-8/Ollie_the_Brave_Baby_Owl.mp3I have successfully created the image and recorded the story as an mp3 file. Both files have been saved in the specified folder with the same name.

Final Answer: The chapter of the baby owl capturing a rodent in the night as his dinner has been created. The corresponding image and audio file have been saved as "Ollie_the_Brave_Baby_Owl.png" and "Ollie_the_Brave_Baby_Owl.mp3" in the folder `tutorial/LLM+Langchain/Week-8`.

Here are the details:
- Image: `tutorial/LLM+Langchain/Week-8/Ollie_the_Brave_Baby_Owl.png`
- Audio: `tutorial/LLM+Langchain/Week-8/Ollie_the_Brave_Baby_Owl.mp3`

> Finished chain.


{'input': 'Create a chapter of a baby owl capturing a rodent in the night as his dinner.\nAfter having the final answer, please create a corresponding image and record the story as an mp3. The saved image (.png) and mp3 (.mp3) should have same name in the folder `tutorial/LLM+Langchain/Week-8',
 'output': 'The chapter of the baby owl capturing a rodent in the night as his dinner has been created. The corresponding image and audio file have been saved as "Ollie_the_Brave_Baby_Owl.png" and "Ollie_the_Brave_Baby_Owl.mp3" in the folder `tutorial/LLM+Langchain/Week-8`.\n\nHere are the details:\n- Image: `tutorial/LLM+Langchain/Week-8/Ollie_the_Brave_Baby_Owl.png`\n- Audio: `tutorial/LLM+Langchain/Week-8/Ollie_the_Brave_Baby_Owl.mp3`'}

In [ ]:
prompt = """
         Assuming that Harry Porter is in the world of Warhammer 40k. He still has his magical power.
         He lives in the lower part of a Hive city. It is about the time for Tithe and the black ship is comming.
         Describe me the fate of Harry Porter. Please keep the darkness of the world view of Warhammer 40k.
        The saved image and mp3 should have same name in the folder `tutorial/LLM+Langchain/Week-8`
        """

agent_executor.invoke({"input": prompt})

## Can we create a story with multiple pages?

I do not know the answer, let me try...

4 pages to save the cost. But it can be extended.

In [40]:
prompt = """
         I want to create an 4 pages story for a child. He likes snow owl.
         For each page, please create a corresponding image and record the story as an mp3.
         After having the final answer, please create a corresponding image and record the story as an mp3. 
         The saved image and mp3 should have same name, following the structure of 
         <Page - idx>, with idx as a number starting from 1, in the folder `tutorial/LLM+Langchain/Week-8`
         """

agent_executor.invoke({"input": prompt})



> Entering new AgentExecutor chain...
To create a 4-page story for a child who likes snow owls, I need to follow these steps:

1. Create the story using the StoryTeller tool.
2. For each page of the story, generate a corresponding image using the ACG characters image generator with GPT-Image-1.
3. Record the story as an mp3 using the Text to Sound (tts) tool.
4. Save the images and mp3 files with the appropriate naming structure.

Let's start by creating the story.

Action: StoryTeller

Action Input: 
```
{
  "input": "Create a 4-page story for a child who likes snow owls."
}
```
**Title: Ollie the Snow Owl's Winter Adventure**

**Page 1:**
Once upon a time, in a snowy forest far, far away, there lived a little snow owl named Ollie. Ollie had the fluffiest white feathers and the biggest, roundest eyes. He loved to explore the winter wonderland around him. Every morning, Ollie would spread his wings and glide over the sparkling snow, looking for new adventures.

**Page 2:**
One chilly

C:\Users\MengChieh\AppData\Local\Temp\ipykernel_13460\3477408663.py:35: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(filename)


tutorial/LLM+Langchain/Week-8/Page-1.mp3Now that we have the audio for Page 1, let's proceed to create the audio for Page 2.

Action: Text to Sound (tts) tool

Action Input: 
```
{
  "text": "One chilly morning, as the sun peeked over the horizon, Ollie noticed something shiny in the snow. He swooped down and found a beautiful, glittering snowflake necklace. 'Wow! This must belong to someone special,' Ollie thought. He decided to find the owner and return the necklace. With a determined hoot, Ollie set off on his quest.",
  "filename": "tutorial/LLM+Langchain/Week-8/Page-2.mp3"
}
```


C:\Users\MengChieh\AppData\Local\Temp\ipykernel_13460\3477408663.py:35: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(filename)


tutorial/LLM+Langchain/Week-8/Page-2.mp3Now that we have the audio for Page 2, let's proceed to create the audio for Page 3.

Action: Text to Sound (tts) tool

Action Input: 
```
{
  "text": "Ollie flew through the forest, asking his friends if they had lost a necklace. He asked Benny the Bunny, who was busy building a snowman. 'No, it's not mine,' said Benny, 'but it sure is pretty!' Next, Ollie asked Sally the Squirrel, who was gathering acorns. 'I haven't seen it before,' said Sally, 'but good luck finding the owner!'",
  "filename": "tutorial/LLM+Langchain/Week-8/Page-3.mp3"
}
```


C:\Users\MengChieh\AppData\Local\Temp\ipykernel_13460\3477408663.py:35: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(filename)


tutorial/LLM+Langchain/Week-8/Page-3.mp3Now that we have the audio for Page 3, let's proceed to create the audio for Page 4.

Action: Text to Sound (tts) tool

Action Input: 
```
{
  "text": "Finally, Ollie reached the tallest tree in the forest, where wise old Grandma Owl lived. 'Grandma Owl, do you know who this necklace belongs to?' Ollie asked. Grandma Owl smiled warmly. 'Oh, Ollie, that necklace belongs to the Snow Queen. She must have dropped it during her nightly flight.' Ollie was thrilled! He flew to the Snow Queen's palace and returned the necklace. The Snow Queen was so grateful that she gave Ollie a special gift—a magical feather that would always guide him home. Ollie thanked her and flew back to his cozy nest, happy and proud of his winter adventure.",
  "filename": "tutorial/LLM+Langchain/Week-8/Page-4.mp3"
}
```


C:\Users\MengChieh\AppData\Local\Temp\ipykernel_13460\3477408663.py:35: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(filename)


tutorial/LLM+Langchain/Week-8/Page-4.mp3I now know the final answer.

Final Answer: The 4-page story for the child who likes snow owls has been created, along with corresponding images and audio recordings for each page. The files are saved in the folder `tutorial/LLM+Langchain/Week-8` with the following structure:

1. **Page 1:**
   - Image: `tutorial/LLM+Langchain/Week-8/Page-1.png`
   - Audio: `tutorial/LLM+Langchain/Week-8/Page-1.mp3`

2. **Page 2:**
   - Image: `tutorial/LLM+Langchain/Week-8/Page-2.png`
   - Audio: `tutorial/LLM+Langchain/Week-8/Page-2.mp3`

3. **Page 3:**
   - Image: `tutorial/LLM+Langchain/Week-8/Page-3.png`
   - Audio: `tutorial/LLM+Langchain/Week-8/Page-3.mp3`

4. **Page 4:**
   - Image: `tutorial/LLM+Langchain/Week-8/Page-4.png`
   - Audio: `tutorial/LLM+Langchain/Week-8/Page-4.mp3`

The story and its corresponding media are now ready for the child to enjoy.

> Finished chain.


{'input': '\n         I want to create an 4 pages story for a child. He likes snow owl.\n         For each page, please create a corresponding image and record the story as an mp3.\n         After having the final answer, please create a corresponding image and record the story as an mp3. \n         The saved image and mp3 should have same name, following the structure of \n         <Page - idx>, with idx as a number starting from 1, in the folder `tutorial/LLM+Langchain/Week-8`\n         ',
 'output': 'The 4-page story for the child who likes snow owls has been created, along with corresponding images and audio recordings for each page. The files are saved in the folder `tutorial/LLM+Langchain/Week-8` with the following structure:\n\n1. **Page 1:**\n   - Image: `tutorial/LLM+Langchain/Week-8/Page-1.png`\n   - Audio: `tutorial/LLM+Langchain/Week-8/Page-1.mp3`\n\n2. **Page 2:**\n   - Image: `tutorial/LLM+Langchain/Week-8/Page-2.png`\n   - Audio: `tutorial/LLM+Langchain/Week-8/Page-2.mp3

## Can we create a story in an interactive way: chat based

-- Rolling back...

In [41]:
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

from langchain.output_parsers import StructuredOutputParser, ResponseSchema

output_response_schemas = [
        ResponseSchema(name="story", description="the story content in the page"),
        ResponseSchema(name="page index", description="The page number of the story"),
    ]

output_parser = StructuredOutputParser.from_response_schemas(output_response_schemas)

output_format_instructions = output_parser.get_format_instructions()


template = """
           Create a story page {idx}, based on the description: {text}

           The answer continues from previous content:
           {context}

           After having the final answer, please create a corresponding image and record the story as an mp3. 
           The saved image and mp3 should have same name, following the structure of 
           <Page - idx>, in the folder `tutorial/LLM+Langchain/Week-8`

           The output should have the following format: {output_format_instruction}
           """

prompt_template = PromptTemplate(template=template,
                                 input_variables=["text", "context", "idx"],
                                 partial_variables={"output_format_instruction": output_format_instructions})

agent_chain = RunnablePassthrough.assign(input=prompt_template)|agent_executor

In [42]:
Q = agent_chain.invoke({"text": "A little cat just woke up in the morning",
                        "context": "The beginning of the story:\n",
                        "idx": str(1)})



> Entering new AgentExecutor chain...
To create the story page, I need to generate the story content first. Then, I will create an image and an audio file for the story. Finally, I will format the output as specified.

Action: StoryTeller

Action Input: "A little cat just woke up in the morning"
**Chapter 1: The Morning Adventure of Whiskers the Cat**

Once upon a time, in a cozy little house at the edge of a friendly forest, there lived a little cat named Whiskers. Whiskers had soft, fluffy fur that was as white as snow, and big, curious eyes that sparkled like twinkling stars.

One sunny morning, Whiskers woke up to the sound of birds chirping happily outside the window. He stretched his tiny paws, arched his back, and let out a big, adorable yawn. "Today is going to be a wonderful day," he thought to himself.

Whiskers jumped out of his comfy bed and padded over to the window. He peeked outside and saw the garden filled with colorful flowers, fluttering butterflies, and a gentle b

C:\Users\MengChieh\AppData\Local\Temp\ipykernel_13460\3477408663.py:35: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(filename)


tutorial/LLM+Langchain/Week-8/Page-1.mp3I have successfully generated the story content, created an image, and recorded the audio file for the story. Now, I will format the output as specified.

Final Answer:
```json
{
  "story": "Once upon a time, in a cozy little house at the edge of a friendly forest, there lived a little cat named Whiskers. Whiskers had soft, fluffy fur that was as white as snow, and big, curious eyes that sparkled like twinkling stars.\n\nOne sunny morning, Whiskers woke up to the sound of birds chirping happily outside the window. He stretched his tiny paws, arched his back, and let out a big, adorable yawn. 'Today is going to be a wonderful day,' he thought to himself.\n\nWhiskers jumped out of his comfy bed and padded over to the window. He peeked outside and saw the garden filled with colorful flowers, fluttering butterflies, and a gentle breeze that made the leaves dance. Whiskers' heart filled with excitement. He loved exploring the garden and making new fri

若是以下步驟失敗，嘗試重新生成。這是大語言模型，沒有保證可以100%產出你希望的格式。我們只能盡可能提高成功輸出的機率。

In [43]:
Q['output']

'```json\n{\n  "story": "Once upon a time, in a cozy little house at the edge of a friendly forest, there lived a little cat named Whiskers. Whiskers had soft, fluffy fur that was as white as snow, and big, curious eyes that sparkled like twinkling stars.\\n\\nOne sunny morning, Whiskers woke up to the sound of birds chirping happily outside the window. He stretched his tiny paws, arched his back, and let out a big, adorable yawn. \'Today is going to be a wonderful day,\' he thought to himself.\\n\\nWhiskers jumped out of his comfy bed and padded over to the window. He peeked outside and saw the garden filled with colorful flowers, fluttering butterflies, and a gentle breeze that made the leaves dance. Whiskers\' heart filled with excitement. He loved exploring the garden and making new friends.\\n\\nAs he trotted down the stairs, he could smell something delicious coming from the kitchen. It was his favorite breakfast—warm milk and a little bowl of crunchy cat treats! Whiskers lapped 

In [44]:
output_parser.parse(Q['output'])

{'story': "Once upon a time, in a cozy little house at the edge of a friendly forest, there lived a little cat named Whiskers. Whiskers had soft, fluffy fur that was as white as snow, and big, curious eyes that sparkled like twinkling stars.\n\nOne sunny morning, Whiskers woke up to the sound of birds chirping happily outside the window. He stretched his tiny paws, arched his back, and let out a big, adorable yawn. 'Today is going to be a wonderful day,' he thought to himself.\n\nWhiskers jumped out of his comfy bed and padded over to the window. He peeked outside and saw the garden filled with colorful flowers, fluttering butterflies, and a gentle breeze that made the leaves dance. Whiskers' heart filled with excitement. He loved exploring the garden and making new friends.\n\nAs he trotted down the stairs, he could smell something delicious coming from the kitchen. It was his favorite breakfast—warm milk and a little bowl of crunchy cat treats! Whiskers lapped up his milk and nibbled

In [45]:
output_parser.parse(Q['output'])['story']

"Once upon a time, in a cozy little house at the edge of a friendly forest, there lived a little cat named Whiskers. Whiskers had soft, fluffy fur that was as white as snow, and big, curious eyes that sparkled like twinkling stars.\n\nOne sunny morning, Whiskers woke up to the sound of birds chirping happily outside the window. He stretched his tiny paws, arched his back, and let out a big, adorable yawn. 'Today is going to be a wonderful day,' he thought to himself.\n\nWhiskers jumped out of his comfy bed and padded over to the window. He peeked outside and saw the garden filled with colorful flowers, fluttering butterflies, and a gentle breeze that made the leaves dance. Whiskers' heart filled with excitement. He loved exploring the garden and making new friends.\n\nAs he trotted down the stairs, he could smell something delicious coming from the kitchen. It was his favorite breakfast—warm milk and a little bowl of crunchy cat treats! Whiskers lapped up his milk and nibbled on his tr

In [46]:
output_parser.parse(Q['output'])['page index']

'1'

### 第二頁

In [47]:
context_list = [output_parser.parse(Q['output'])['story']]
print(context_list)

["Once upon a time, in a cozy little house at the edge of a friendly forest, there lived a little cat named Whiskers. Whiskers had soft, fluffy fur that was as white as snow, and big, curious eyes that sparkled like twinkling stars.\n\nOne sunny morning, Whiskers woke up to the sound of birds chirping happily outside the window. He stretched his tiny paws, arched his back, and let out a big, adorable yawn. 'Today is going to be a wonderful day,' he thought to himself.\n\nWhiskers jumped out of his comfy bed and padded over to the window. He peeked outside and saw the garden filled with colorful flowers, fluttering butterflies, and a gentle breeze that made the leaves dance. Whiskers' heart filled with excitement. He loved exploring the garden and making new friends.\n\nAs he trotted down the stairs, he could smell something delicious coming from the kitchen. It was his favorite breakfast—warm milk and a little bowl of crunchy cat treats! Whiskers lapped up his milk and nibbled on his t

In [48]:
Q_2 = agent_chain.invoke({"text": "Whisker found a dove and wanted to hunt it down!",
                          "context": ":\n".join(context_list),
                          "idx": str(2)})



> Entering new AgentExecutor chain...
To create the second page of the story, I need to continue from where the previous content left off and incorporate the new description about Whiskers finding a dove and wanting to hunt it down. After crafting the story, I will generate an image and an audio file for the story.

Let's start by creating the continuation of the story.

Thought: I need to create the continuation of the story based on the given description.

Action: StoryTeller

Action Input: 
```
{
  "input": "Whisker found a dove and wanted to hunt it down!\n\nOnce upon a time, in a cozy little house at the edge of a friendly forest, there lived a little cat named Whiskers. Whiskers had soft, fluffy fur that was as white as snow, and big, curious eyes that sparkled like twinkling stars.\n\nOne sunny morning, Whiskers woke up to the sound of birds chirping happily outside the window. He stretched his tiny paws, arched his back, and let out a big, adorable yawn. 'Today is going to be

C:\Users\MengChieh\AppData\Local\Temp\ipykernel_13460\3477408663.py:35: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(filename)


tutorial/LLM+Langchain/Week-8/Page-2.mp3I have successfully created the continuation of the story, generated an image, and recorded the story as an mp3 file. Here is the final output:

```json
{
	"story": "Whiskers and Thumper had become the best of friends, and every day was filled with new adventures. One sunny afternoon, as they were playing near the big oak tree, Whiskers noticed a beautiful white dove perched on a branch. The dove's feathers glistened in the sunlight, and it cooed softly, adding to the peaceful ambiance of the garden.\n\nWhiskers' natural instincts kicked in, and he crouched low, his eyes fixed on the dove. 'I wonder if I can catch it,' he thought, his tail twitching with excitement. He slowly crept closer, trying to be as quiet as possible.\n\nThumper noticed Whiskers' change in behavior and hopped over to him. 'What are you doing, Whiskers?' he asked, his nose twitching with curiosity.\n\n'I'm going to catch that dove,' Whiskers whispered, his eyes never leaving

In [49]:
context_list

["Once upon a time, in a cozy little house at the edge of a friendly forest, there lived a little cat named Whiskers. Whiskers had soft, fluffy fur that was as white as snow, and big, curious eyes that sparkled like twinkling stars.\n\nOne sunny morning, Whiskers woke up to the sound of birds chirping happily outside the window. He stretched his tiny paws, arched his back, and let out a big, adorable yawn. 'Today is going to be a wonderful day,' he thought to himself.\n\nWhiskers jumped out of his comfy bed and padded over to the window. He peeked outside and saw the garden filled with colorful flowers, fluttering butterflies, and a gentle breeze that made the leaves dance. Whiskers' heart filled with excitement. He loved exploring the garden and making new friends.\n\nAs he trotted down the stairs, he could smell something delicious coming from the kitchen. It was his favorite breakfast—warm milk and a little bowl of crunchy cat treats! Whiskers lapped up his milk and nibbled on his t

In [50]:
output_parser.parse(Q_2['output'])['story']

OutputParserException: Got invalid JSON object. Error: Expecting value: line 1 column 1 (char 0)
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 

### okay, it looks fine, let us see how to make it a interactive

In [ ]:
# "前情提要"
context_list = []

# 頁面起始
idx = 1

while True:
    if len(context_list) == 0:
        context = "The beginning of the story:\n"
    else:
        context = "\n".join(context_list)

    text = input("請輸入故事內容: 若想要結束 請輸入 `QUIT`")

    if text == "QUIT":
        break
    
    Q = agent_chain.invoke({"text": text,
                            "context": context,
                            "idx": str(idx)})

    story = output_parser.parse(Q['output'])['story']
    
    # 下一頁
    idx += 1

    context_list.append(output_parser.parse(Q['output'])['story'])